In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.float_format', '{:.11e}'.format)
from sklearn.ensemble import RandomForestClassifier
from pathlib import Path
np.random.seed(0)

In [2]:
from combat.pycombat import pycombat

### Clean the data in each dataframe and add the disease status of each sample  

In [188]:
df_feng = pd.read_csv(datadir/'centrifuge_abundance_feng.csv',header=0).reset_index(drop=True)   #Read in the file
df_feng.rename(columns={df_feng.columns[0]: 'Sample'}, inplace=True)  #Name the Sample Column (lets keep it for record keeping)
Feng_df=df_feng.set_index('Sample')   #Index the Sample Column

# Let's do it for Feng Read in the metadata and clean it up 
feng_meta = pd.read_csv(datadir/'metadata_feng.csv',header=0).reset_index(drop=True)
feng_meta = feng_meta.rename(columns={feng_meta.columns[3]:'X3'})  # needs the name X3 since it is currently nameless
#Change carcinoma to CRC and controls to CTC
feng_meta['X3'] = feng_meta['X3'].replace({'carcinoma': 'CRC', 'controls': 'CTC'})
feng_meta['status'] = feng_meta['X3']
Feng_df = Feng_df.reset_index()
Feng_df['status'] = feng_meta['X3']
# we need to exclude the adenoma samples
Feng_df = Feng_df[Feng_df['status']!='advanced adenoma']

Feng_df


,Sample,24,162,195,197,199,200,204,210,253,...,2597770,2599293,2599308,2603276,2603292,2603296,2605619,2605620,2609668,status
0,ERR688505,0.00000000000e+00,0.00000000000e+00,5.33155000000e-04,1.82055000000e-05,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,6.97045000000e-07,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,CRC
1,ERR688506,0.00000000000e+00,0.00000000000e+00,3.75351000000e-05,1.01813000000e-05,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,1.99087000000e-55,0.00000000000e+00,1.97062000000e-58,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,CTC
2,ERR688507,0.00000000000e+00,0.00000000000e+00,1.92533000000e-04,1.61290000000e-05,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,6.91492000000e-06,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,CTC
4,ERR688509,0.00000000000e+00,0.00000000000e+00,1.83838000000e-04,7.84099000000e-06,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,CTC
5,ERR688510,0.00000000000e+00,0.00000000000e+00,1.53972000000e-05,1.91006000000e-05,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,CTC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,ERR710427,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,1.02657000000e-06,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,CTC
151,ERR710428,0.00000000000e+00,0.00000000000e+00,1.95856000000e-04,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,CTC
152,ERR710429,0.00000000000e+00,0.00000000000e+00,3.11318000000e-04,7.44415000000e-05,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,CTC
153,ERR710430,0.00000000000e+00,3.66608000000e-07,3.30482000000e-05,6.92511000000e-06,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,...,5.61740000000e-07,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,CTC


In [81]:
df_yu = pd.read_csv(datadir/'centrifuge_abundance_yu.csv',header=0).reset_index(drop=True)
df_yu.rename(columns={df_yu.columns[0]: 'Sample'}, inplace=True)  #Name the Sample Column (lets keep it for record keeping)
Yu_df=df_yu.set_index('Sample')   #Index the Sample Column

#Let's do it for Yu
yu_meta = pd.read_csv(datadir/'metadata_yu.csv',header=None).reset_index(drop=True)
Yu_df = Yu_df.reset_index()
Yu_df['status'] = yu_meta[yu_meta.columns[15]]
Yu_df.head()

,Sample,159,162,195,197,199,200,204,210,253,...,2594789,2597659,2597770,2599308,2599640,2601898,2603296,2605619,2605620,status
0,ERR1018185,0.00000000000e+00,0.00000000000e+00,1.53055000000e-03,6.07131000000e-05,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,...,1.85252000000e-04,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,4.51834000000e-206,0.00000000000e+00,3.36375000000e-05,CRC
1,ERR1018186,0.00000000000e+00,0.00000000000e+00,7.00093000000e-04,1.93446000000e-04,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,...,3.53183000000e-05,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,CRC
2,ERR1018187,0.00000000000e+00,4.29069000000e-08,1.42968000000e-04,5.75686000000e-05,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,...,1.09821000000e-04,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,1.87935000000e-07,CRC
3,ERR1018188,0.00000000000e+00,7.19795000000e-07,4.59199000000e-04,1.93690000000e-04,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,...,8.11997000000e-05,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,7.29764000000e-07,0.00000000000e+00,3.90951000000e-07,CRC
4,ERR1018189,0.00000000000e+00,0.00000000000e+00,3.93074000000e-04,2.51274000000e-05,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,...,1.46185000000e-03,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,6.12226000000e-212,0.00000000000e+00,0.00000000000e+00,CRC


In [84]:
# Read in the abundance dataset 
df_hannigan = pd.read_csv(datadir/'centrifuge_abundance_hannigan.csv',header=0).reset_index(drop=True)
df_hannigan.rename(columns={df_feng.columns[0]: 'Sample'}, inplace=True)
Hannigan_df=df_feng.set_index('Sample')
#read in metadata, name the column that holds the disease status and append disease status to abundance dataset
hann_meta = pd.read_csv(datadir/'metadata_hannigan.csv',header = None).reset_index(drop=True)
hann_meta = hann_meta.rename(columns={hann_meta.columns[4]:'X4'})
hann_meta = hann_meta.replace({'Cancer':'CRC','Healthy':'CTC'})
hann_meta = hann_meta[(hann_meta['X4'] != 'SRR5665060') & (hann_meta['X4'] != 'SRR5665121')]  # we need to get rid of these samples 
Hannigan_df = Hannigan_df.reset_index()
Hannigan_df['status'] = hann_meta['X4'] 
Hannigan_df = Hannigan_df[Hannigan_df['status']!='Adenoma'] ##make a abundance dataset without the Adenoma samples since we only want cancer and control samples


In [85]:
Hannigan_df.head()

,Sample,24,162,195,197,199,200,204,210,253,...,2597770,2599293,2599308,2603276,2603292,2603296,2605619,2605620,2609668,status
0,ERR688505,0.00000000000e+00,0.00000000000e+00,5.33155000000e-04,1.82055000000e-05,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,6.97045000000e-07,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,CRC
1,ERR688506,0.00000000000e+00,0.00000000000e+00,3.75351000000e-05,1.01813000000e-05,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,1.99087000000e-55,0.00000000000e+00,1.97062000000e-58,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,CRC
2,ERR688507,0.00000000000e+00,0.00000000000e+00,1.92533000000e-04,1.61290000000e-05,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,6.91492000000e-06,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,CRC
3,ERR688508,0.00000000000e+00,0.00000000000e+00,4.55617000000e-04,2.76764000000e-04,3.84208000000e-07,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,2.77079000000e-07,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,CRC
4,ERR688509,0.00000000000e+00,0.00000000000e+00,1.83838000000e-04,7.84099000000e-06,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,CRC


In [95]:
#Read in the abundance dataset 
df_vogtmann = pd.read_csv(datadir/'centrifuge_abundance_vogtmann.csv',header=0).reset_index(drop=True)                                                                                
df_vogtmann.rename(columns={df_vogtmann.columns[0]: 'Sample'}, inplace=True)  #Name the Sample Column (lets keep it for record keeping)
Votmann_df=df_vogtmann.set_index('Sample')   #Index the Sample Column
#Read in Metadata, rename how the disease status is written in dataset (1->CRC and 0->CTC),add disease status to the abundance dataset
vog_meta = pd.read_csv(datadir/'metadata_vogtmann.csv',header = 0).reset_index(drop =True)
vog_meta['casectl'] = vog_meta['casectl'].replace({1:'CRC',0:'CTC'})
Votmann_df= Votmann_df.reset_index()
Votmann_df['status'] = vog_meta['casectl']

In [119]:
df_zeller = pd.read_csv(datadir/'centrifuge_abundance_zeller.csv',header=0).reset_index(drop=True)
df_zeller.rename(columns={df_zeller.columns[0]: 'Sample'}, inplace=True)  #Name the Sample Column (lets keep it for record keeping)
Zeller_df=df_zeller.set_index('Sample') #Index the Sample Column

Zeller_df = Zeller_df.sort_values(by='Sample')
Zeller_df = Zeller_df.reset_index()


zeller_meta = pd.read_csv(datadir/'Zeller_CRC.csv')
zeller_meta = zeller_meta.sort_values(by='No').reset_index(drop=True)
zeller_meta['study_condition'] = zeller_meta['study_condition'].replace({'control': 'CTC'}) 
Zeller_df['status']=zeller_meta['study_condition']
Zeller_df

,Sample,24,63,159,162,192,195,197,199,200,...,2597770,2599308,2601898,2603277,2603296,2605620,2607911,2608254,2609668,status
0,1,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,6.14405000000e-04,8.54424000000e-05,1.20528000000e-06,0.00000000000e+00,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,1.33525000000e-06,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,CTC
1,10,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,2.02403000000e-04,5.66455000000e-06,0.00000000000e+00,0.00000000000e+00,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,1.07214000000e-05,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,CTC
2,100,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,1.32894000000e-07,0.00000000000e+00,8.92398000000e-05,1.65682000000e-05,0.00000000000e+00,0.00000000000e+00,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,5.08564000000e-04,6.49332000000e-07,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,CRC
3,101,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,3.17741000000e-04,3.24538000000e-05,4.78299000000e-07,0.00000000000e+00,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,7.63054000000e-06,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,CRC
4,102,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,9.71932000000e-04,1.21706000000e-04,0.00000000000e+00,0.00000000000e+00,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,6.60522000000e-293,2.23156000000e-136,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,CRC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,DE-098,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,2.92436000000e-07,0.00000000000e+00,4.53032000000e-04,2.96260000000e-05,3.61992000000e-06,0.00000000000e+00,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,9.18760000000e-07,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,CRC
180,DE-099,0.00000000000e+00,0.00000000000e+00,1.83409000000e-120,0.00000000000e+00,0.00000000000e+00,5.09838000000e-05,5.03474000000e-06,0.00000000000e+00,0.00000000000e+00,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,3.56570000000e-07,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,CRC
181,DE-100,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,1.07597000000e-04,3.45130000000e-05,0.00000000000e+00,0.00000000000e+00,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,CRC
182,DE-101,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,1.43092000000e-04,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,CRC


In [150]:
#Read in the abundance dataset
df_thomas = pd.read_csv(datadir/'centrifuge_abundance_thomas.csv',header=0)
Thomas_df = df_thomas.rename(columns={df_thomas.columns[0]: 'Sample'})  #Name the Sample Column (lets keep it for record keeping)
#Read in the metadata
thomas_meta = pd.read_csv(datadir/'metadata_thomas.csv',header=None)
thomas_meta= thomas_meta.rename(columns={thomas_meta.columns[0]:'X0'})  #This dataset does not have a header so we will name the column with the disease status
#Append the disease status to the abundance dataset
Thomas_df['status'] = thomas_meta['X0']
Thomas_df
#thomas_meta

,Sample,24,159,162,195,197,199,200,204,210,...,2597701,2597770,2599308,2602769,2603277,2603296,2605619,2605620,2608254,status
0,SRR6915091,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,4.43194000000e-07,0.00000000000e+00,CRC
1,SRR6915092,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,1.55308000000e-04,2.01714000000e-05,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,CTR
2,SRR6915093,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,1.46181000000e-05,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,CRC
3,SRR6915094,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,9.52111000000e-07,8.17117000000e-08,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,3.97285000000e-06,0.00000000000e+00,4.66258000000e-08,0.00000000000e+00,CRC
4,SRR6915095,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,6.06577000000e-07,3.74093000000e-165,0.00000000000e+00,0.00000000000e+00,CRC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,SRR6915220,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,1.31021000000e-04,2.39129000000e-05,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,3.35769000000e-06,0.00000000000e+00,CRC
109,SRR6915222,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,1.61171000000e-05,9.68377000000e-08,3.12211000000e-07,1.66331000000e-07,0.00000000000e+00,0.00000000000e+00,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,4.23601000000e-231,8.66170000000e-09,0.00000000000e+00,0.00000000000e+00,CTR
110,SRR6915223,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,9.38274000000e-05,8.77331000000e-05,9.35252000000e-07,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,CRC
111,SRR6915225,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,3.53814000000e-04,4.15969000000e-05,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,...,0.00000000000e+00,3.35660000000e-07,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,3.55235000000e-07,0.00000000000e+00,CTR


### Make a huge concatenated dataset of all datasets(without status) + concatenated dataset of disease status

In [195]:
#Every dataframe made is already sorted in numberical order for each species(column-wise)
#So lets Grab all the data points in each dataset but not the status
Feng = Feng_df.iloc[:,1:-1]
Yu = Yu_df.iloc[:,1:-1]
Hannigan = Hannigan_df.iloc[:,1:-1]
Vogtmann = Votmann_df.iloc[:,1:-1]
Zeller = Zeller_df.iloc[:,1:-1]
Thomas = Thomas_df.iloc[:,1:-1]

# Lets concatenate the dataframes
Abundance_df = pd.concat([Feng, Yu, Hannigan, Vogtmann, Zeller, Thomas])
Abundance_df = Abundance_df.reset_index(drop=True)
Abundance_df = Abundance_df.fillna(0)

# Make Status dataframe
Feng_status = Feng_df.iloc[:,-1]
Yu_status = Yu_df.iloc[:,-1]
Hannigan_status = Hannigan_df.iloc[:,-1]
Vogtmann_status = Votmann_df.iloc[:,-1]
Zeller_status = Zeller_df.iloc[:,-1]
Thomas_status = Thomas_df.iloc[:,-1]
Status_df = pd.concat([Feng_status,Yu_status,Hannigan_status,Vogtmann_status,Zeller_status,Thomas_status])
Status_df = Status_df.reset_index(drop=True)

# Ineed to generate code for training, test and then do batch for training =0 and batch for test = 1 and then perform pyCombat. We need to figure out if we are using Kfold cr
#cross valid or just train test split

In [197]:
Yu_df.iloc[:,-1]

0      CRC
1      CRC
2      CRC
3      CRC
4      CRC
      ... 
123    CTR
124    CTR
125    CRC
126    CRC
127    CRC
Name: status, Length: 128, dtype: object

In [ ]:
# Add batch variable
#separate data into training and test 1st co 
training['batch'] = 0
test['batch'] = 1
data_norm = pycombat
    
# Combine training and test data for ComBat normalization
data = pd.concat([training, test])
batch = data['batch']
data_norm = data.drop(columns='batch').T
# Apply ComBat normalization 
data_norm = pycombat(data_norm, batch)


In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RepeatedStratifiedKFold, GridSearchCV
from sklearn.metrics import roc_auc_score





In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RepeatedStratifiedKFold, GridSearchCV
from sklearn.metrics import roc_auc_score

def rf_combat(training, training_status, test, test_status, num_trees, num_rep):
    
    
    # Ensure columns in test match those in training
    missing_columns = set(training.columns) - set(test.columns)
    for col in missing_columns:
        test[col] = 0
    test = test[training.columns]

    # Add batch variable
    training['batch'] = 0
    test['batch'] = 1
    
    # Combine training and test data for ComBat normalization
    data = pd.concat([training, test])
    batch = data['batch']
    data_norm = data.drop(columns='batch').T
    # Apply ComBat normalization - you will need to implement ComBat or use an appropriate library
    # e.g., if using the PyCombat library: data_norm = pycombat(data_norm, batch)
    
    training_norm = data_norm.iloc[:, :len(training)].T
    training_norm['status'] = training_status
    test_norm = data_norm.iloc[:, len(training):].T
    test_norm['status'] = test_status

    # Train Random Forest model
    rf = RandomForestClassifier(n_estimators=num_trees)
    param_grid = {'max_depth': [10, 20, 30]}
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=num_rep)
    grid_search = GridSearchCV(rf, param_grid, scoring='roc_auc', cv=cv)
    grid_search.fit(training_norm.drop(columns='status'), training_norm['status'])

    # Predict and compute AUC for test data
    pred = grid_search.predict_proba(test_norm.drop(columns='status'))[:, 1]
    auc = roc_auc_score(test_norm['status'], pred)

    return auc


In [157]:
#df=Zeller_df.reindex(sorted(Zeller_df.columns, key=lambda x :int(x)), axis=1)
Zeller_df[Zeller_df.columns[1:]]

,24,63,159,162,192,195,197,199,200,204,...,2597770,2599308,2601898,2603277,2603296,2605620,2607911,2608254,2609668,status
0,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,6.14405000000e-04,8.54424000000e-05,1.20528000000e-06,0.00000000000e+00,0.00000000000e+00,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,1.33525000000e-06,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,CTC
1,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,2.02403000000e-04,5.66455000000e-06,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,1.07214000000e-05,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,CTC
2,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,1.32894000000e-07,0.00000000000e+00,8.92398000000e-05,1.65682000000e-05,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,5.08564000000e-04,6.49332000000e-07,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,CRC
3,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,3.17741000000e-04,3.24538000000e-05,4.78299000000e-07,0.00000000000e+00,0.00000000000e+00,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,7.63054000000e-06,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,CRC
4,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,9.71932000000e-04,1.21706000000e-04,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,6.60522000000e-293,2.23156000000e-136,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,CRC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,2.92436000000e-07,0.00000000000e+00,4.53032000000e-04,2.96260000000e-05,3.61992000000e-06,0.00000000000e+00,7.85248000000e-05,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,9.18760000000e-07,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,CRC
180,0.00000000000e+00,0.00000000000e+00,1.83409000000e-120,0.00000000000e+00,0.00000000000e+00,5.09838000000e-05,5.03474000000e-06,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,3.56570000000e-07,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,CRC
181,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,1.07597000000e-04,3.45130000000e-05,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,CRC
182,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,1.43092000000e-04,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,CRC


In [159]:
Votmann_df

,Sample,63,159,162,195,197,198,199,200,204,...,2592352,2592354,2592355,2594269,2594473,2594789,2601898,2603296,2605620,status
0,MMRS11288076ST,0.00000000000e+00,0.00000000000e+00,1.54464000000e-07,2.36499000000e-04,7.29568000000e-04,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,5.58667000000e-05,0.00000000000e+00,2.10194000000e-288,0.00000000000e+00,CTC
1,MMRS11932626ST,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,3.71187000000e-04,2.26994000000e-04,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,8.65502000000e-04,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,CTC
2,MMRS12272136ST,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,5.78023000000e-04,5.80794000000e-04,0.00000000000e+00,4.78547000000e-07,0.00000000000e+00,0.00000000000e+00,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,1.19753000000e-04,0.00000000000e+00,2.36877000000e-32,0.00000000000e+00,CRC
3,MMRS14379078ST,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,2.82139000000e-04,7.76475000000e-05,0.00000000000e+00,3.06115000000e-07,0.00000000000e+00,0.00000000000e+00,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,5.92440000000e-05,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,CRC
4,MMRS14602194ST,0.00000000000e+00,0.00000000000e+00,4.14382000000e-08,1.21646000000e-04,3.24307000000e-06,0.00000000000e+00,3.40171000000e-07,0.00000000000e+00,0.00000000000e+00,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,1.31975000000e-03,5.23262000000e-225,0.00000000000e+00,0.00000000000e+00,CRC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,MMRS96422329ST,0.00000000000e+00,0.00000000000e+00,8.89905000000e-08,1.21676000000e-04,2.85284000000e-05,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,4.71437000000e-04,0.00000000000e+00,6.78557000000e-52,0.00000000000e+00,CRC
100,MMRS97052561ST,0.00000000000e+00,0.00000000000e+00,1.34057000000e-07,4.96187000000e-04,5.05719000000e-05,0.00000000000e+00,6.58466000000e-07,0.00000000000e+00,4.26510000000e-07,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,1.28308000000e-03,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,CRC
101,MMRS97307143ST,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,8.20710000000e-04,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,5.16825000000e-04,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,CTC
102,MMRS97327636ST,0.00000000000e+00,0.00000000000e+00,1.39482000000e-06,8.17918000000e-04,2.80506000000e-04,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,4.17609000000e-03,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,CRC


In [154]:
# Suppose 'Zeller_df' is your DataFrame
# Sort columns numerically, excluding 'Sample' column
columns_to_sort = [col for col in Zeller_df.columns if col != 'Sample']
sorted_columns = sorted(columns_to_sort, key=lambda x: int(x))
sorted_columns_with_sample = ['Sample'] + sorted_columns  # Add 'Sample' column back
df = Zeller_df.reindex(columns=sorted_columns_with_sample)
df

ValueError: invalid literal for int() with base 10: 'status'

In [161]:
Zeller_df

,Sample,24,63,159,162,192,195,197,199,200,...,2597770,2599308,2601898,2603277,2603296,2605620,2607911,2608254,2609668,status
0,1,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,6.14405000000e-04,8.54424000000e-05,1.20528000000e-06,0.00000000000e+00,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,1.33525000000e-06,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,CTC
1,10,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,2.02403000000e-04,5.66455000000e-06,0.00000000000e+00,0.00000000000e+00,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,1.07214000000e-05,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,CTC
2,100,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,1.32894000000e-07,0.00000000000e+00,8.92398000000e-05,1.65682000000e-05,0.00000000000e+00,0.00000000000e+00,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,5.08564000000e-04,6.49332000000e-07,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,CRC
3,101,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,3.17741000000e-04,3.24538000000e-05,4.78299000000e-07,0.00000000000e+00,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,7.63054000000e-06,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,CRC
4,102,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,9.71932000000e-04,1.21706000000e-04,0.00000000000e+00,0.00000000000e+00,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,6.60522000000e-293,2.23156000000e-136,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,CRC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,DE-098,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,2.92436000000e-07,0.00000000000e+00,4.53032000000e-04,2.96260000000e-05,3.61992000000e-06,0.00000000000e+00,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,9.18760000000e-07,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,CRC
180,DE-099,0.00000000000e+00,0.00000000000e+00,1.83409000000e-120,0.00000000000e+00,0.00000000000e+00,5.09838000000e-05,5.03474000000e-06,0.00000000000e+00,0.00000000000e+00,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,3.56570000000e-07,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,CRC
181,DE-100,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,1.07597000000e-04,3.45130000000e-05,0.00000000000e+00,0.00000000000e+00,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,CRC
182,DE-101,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,1.43092000000e-04,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,...,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,0.00000000000e+00,CRC
